<a href="https://colab.research.google.com/github/KyunghoonJeon/AI-for-ML/blob/main/w5_prac_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis/ch_03

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/long_data.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['station'].unique()

In [ ]:
df['datatype'].unique()

In [ ]:
long_df = pd.read_csv(
    'data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

In [ ]:
long_df.set_index('date').head(6)

In [ ]:
long_df.set_index('date').head(6).T

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head()

In [ ]:
pivoted_df.describe()

In [ ]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)
pivoted_df.head()

In [ ]:
pivoted_df['temp_F']['TMIN'].head()

In [ ]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

In [ ]:
multi_index_df.head()

In [ ]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

In [ ]:
wide_df = pd.read_csv('data/wide_data.csv')
wide_df.head()

In [ ]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

In [ ]:
wide_df.set_index('date', inplace=True)
wide_df.head()

In [ ]:
stacked_series = wide_df.stack()
stacked_series.head()

In [ ]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

In [ ]:
stacked_df.head().index

In [ ]:
stacked_df.index.names

In [ ]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names

In [ ]:
stacked_df.head()

In [ ]:
df1 = pd.read_csv('data/dirty_data.csv')

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
contain_nulls = df1[
    df1.SNOW.isna() | df1.SNWD.isna() | df1.TOBS.isna()
    | df1.WESF.isna() | df1.inclement_weather.isna()
]
contain_nulls.shape[0]

In [ ]:
contain_nulls.head(10)

In [ ]:
df1[df1.inclement_weather.isna()].shape[0]

In [ ]:
df1[df1.SNWD.isin([-np.inf, np.inf])].shape[0]

In [ ]:
def get_inf_count(df):
    """Find the number of inf/-inf values per column in the dataframe"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }

get_inf_count(df1)

In [ ]:
pd.DataFrame({
    'np.inf Snow Depth': df1[df1.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth': df1[df1.SNWD == -np.inf].SNOW.describe()
}).T

In [ ]:
df.describe(include='object')

In [ ]:
df1[df1.duplicated()].shape[0]

In [ ]:
df1.duplicated()

In [ ]:
df1[df1.duplicated()].head()

In [ ]:
df1[df1.WESF.notna()].station.unique()

In [ ]:
# 1. make the date a datetime
df.date = pd.to_datetime(df1.date)

# 2. save this information for later
station_qm_wesf = df1[df1.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. sort ? to the bottom
df1.sort_values('station', ascending=False, inplace=True)

# 4. drop duplicates based on the date column keeping the first occurrence 
# which will be the valid station if it has data
df1_deduped = df1.drop_duplicates('date')

# 5. remove the station column because we are done with it
df1_deduped = df1_deduped.drop(columns='station').set_index('date').sort_index()

# 6. take valid station's WESF and fall back on station ? if it is null
df1_deduped = df1_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df1_deduped.shape

In [ ]:
df1_deduped.head()